In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
import os
import subprocess

# os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars elasticsearch-spark-30_2.12-8.8.0.jar pyspark-shell'

os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars elasticsearch-spark-30_2.12-8.8.0.jar --packages org.elasticsearch:elasticsearch-hadoop:8.8.0 pyspark-shell'


# spark = SparkSession.builder.appName("recommendation_system").config("spark.driver.memory", "3g").getOrCreate()
spark = SparkSession.builder.appName("recommendation_system").getOrCreate()

spark

23/06/25 19:30:40 WARN Utils: Your hostname, ilaria-MS-7B94 resolves to a loopback address: 127.0.1.1; using 157.27.31.4 instead (on interface enp4s0)
23/06/25 19:30:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/student/.ivy2/cache
The jars for the packages stored in: /home/student/.ivy2/jars


:: loading settings :: url = jar:file:/home/student/Projects/bg/.conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


org.elasticsearch#elasticsearch-hadoop added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6e2960b6-755b-4ab4-b34d-3a6c8edc2c70;1.0
	confs: [default]
	found org.elasticsearch#elasticsearch-hadoop;8.8.0 in central
	found commons-logging#commons-logging;1.1.1 in central
	found commons-codec#commons-codec;1.4 in central
	found javax.xml.bind#jaxb-api;2.3.1 in central
	found org.apache.hive#hive-service;3.1.2 in central
	found org.apache.hive#hive-exec;3.1.2 in central
	found org.apache.hive#hive-metastore;3.1.2 in central
	found org.apache.pig#pig;0.15.0 in central
	found org.apache.spark#spark-yarn_2.11;2.3.0 in central
	found org.scala-lang#scala-reflect;2.11.12 in central
	found org.apache.storm#storm-core;1.0.6 in central
	found org.apache.hadoop#hadoop-client;3.1.2 in central
	found org.apache.hadoop#hadoop-common;3.1.2 in central
	found org.apache.hadoop#hadoop-mapreduce-client-core;3.1.2 in central
	found joda-time#joda-time;2.9.3 in central
:: r

In [2]:
BASE_DATA_FOLDER = "ml-latest-small/"
if not os.path.exists(os.path.join('stats', BASE_DATA_FOLDER)):
    os.makedirs(os.path.join('stats', BASE_DATA_FOLDER))

save_stats = False  # set this flag to true to save execution statistics about CPU and RAM usage

In [3]:
if save_stats:
    proc_ok = False
    if not os.path.exists(os.path.join('stats', BASE_DATA_FOLDER, '00_load_movies' + '.csv')):
        stats = subprocess.Popen(['./monitor.sh', BASE_DATA_FOLDER, '00_load_movies'])
        proc_ok = True
    from pyspark.sql.types import *

movies_data = spark.read.option("header", True).csv(BASE_DATA_FOLDER + "movies.csv")
movies_data = movies_data.dropDuplicates(['movieId'])
movies_data = movies_data.withColumn("genres", split(col("genres"), "\\|"))
movies_data.limit(3).toPandas()
if save_stats and proc_ok: stats.kill()

23/06/25 19:30:49 INFO Version: Elasticsearch Hadoop v8.8.0 [9445cb1845]
23/06/25 19:30:49 INFO InMemoryFileIndex: It took 30 ms to list leaf files for 1 paths.
23/06/25 19:30:49 INFO InMemoryFileIndex: It took 1 ms to list leaf files for 1 paths.
23/06/25 19:30:51 INFO FileSourceStrategy: Pushed Filters: 
23/06/25 19:30:51 INFO FileSourceStrategy: Post-Scan Filters: (length(trim(value#0, None)) > 0)
23/06/25 19:30:51 INFO CodeGenerator: Code generated in 137.136362 ms
23/06/25 19:30:51 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 206.8 KiB, free 434.2 MiB)
23/06/25 19:30:51 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 36.3 KiB, free 434.2 MiB)
23/06/25 19:30:51 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on brainavlab1.di.univr.it:44791 (size: 36.3 KiB, free: 434.4 MiB)
23/06/25 19:30:51 INFO SparkContext: Created broadcast 0 from csv at NativeMethodAccessorImpl.java:0
23/06/25 19:30:51 INFO FileSo

In [4]:
movies_data.groupBy("genres").count().orderBy(col('count').desc()).show(100)

23/06/25 19:30:55 INFO FileSourceStrategy: Pushed Filters: 
23/06/25 19:30:55 INFO FileSourceStrategy: Post-Scan Filters: 
23/06/25 19:30:55 INFO MemoryStore: Block broadcast_6 stored as values in memory (estimated size 206.8 KiB, free 433.4 MiB)
23/06/25 19:30:55 INFO MemoryStore: Block broadcast_6_piece0 stored as bytes in memory (estimated size 36.3 KiB, free 433.3 MiB)
23/06/25 19:30:55 INFO BlockManagerInfo: Added broadcast_6_piece0 in memory on brainavlab1.di.univr.it:44791 (size: 36.3 KiB, free: 434.2 MiB)
23/06/25 19:30:55 INFO SparkContext: Created broadcast 6 from showString at NativeMethodAccessorImpl.java:0
23/06/25 19:30:55 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
23/06/25 19:30:55 INFO DAGScheduler: Registering RDD 23 (showString at NativeMethodAccessorImpl.java:0) as input to shuffle 1
23/06/25 19:30:55 INFO DAGScheduler: Got map stage job 3 (showString at NativeMethodAccessorImpl

+--------------------+-----+
|              genres|count|
+--------------------+-----+
|             [Drama]| 1053|
|            [Comedy]|  946|
|     [Comedy, Drama]|  435|
|   [Comedy, Romance]|  363|
|    [Drama, Romance]|  349|
|       [Documentary]|  339|
|[Comedy, Drama, R...|  276|
|   [Drama, Thriller]|  168|
|            [Horror]|  167|
|  [Horror, Thriller]|  135|
|      [Crime, Drama]|  134|
|[Crime, Drama, Th...|  125|
|        [Drama, War]|  114|
|     [Comedy, Crime]|  101|
|    [Action, Comedy]|   92|
|          [Thriller]|   84|
|  [Children, Comedy]|   74|
|    [Comedy, Horror]|   69|
|[Action, Adventur...|   66|
|[Action, Crime, T...|   66|
|     [Action, Drama]|   62|
|[Action, Crime, D...|   61|
|            [Action]|   60|
|  [Action, Thriller]|   60|
|    [Horror, Sci-Fi]|   53|
|[Action, Crime, D...|   50|
|   [Crime, Thriller]|   45|
|    [Drama, Musical]|   44|
|[Action, Drama, T...|   43|
|[Action, Sci-Fi, ...|   43|
|[Horror, Mystery,...|   42|
|[Action, Come

23/06/25 19:30:56 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
23/06/25 19:30:56 INFO DAGScheduler: Got job 5 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
23/06/25 19:30:56 INFO DAGScheduler: Final stage: ResultStage 9 (showString at NativeMethodAccessorImpl.java:0)
23/06/25 19:30:56 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 8)
23/06/25 19:30:56 INFO DAGScheduler: Missing parents: List()
23/06/25 19:30:56 INFO DAGScheduler: Submitting ResultStage 9 (MapPartitionsRDD[32] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
23/06/25 19:30:56 INFO MemoryStore: Block broadcast_9 stored as values in memory (estimated size 49.4 KiB, free 433.2 MiB)
23/06/25 19:30:56 INFO MemoryStore: Block broadcast_9_piece0 stored as bytes in memory (estimated size 22.8 KiB, free 433.2 MiB)
23/06/25 19:30:56 INFO BlockManagerInfo: Added broadcast_9_piece0 in memory on brainavlab1.di.univr.it:44791 

In [5]:
if save_stats:
    proc_ok = False
    if not os.path.exists(os.path.join('stats', BASE_DATA_FOLDER, '01_load_ratings' + '.csv')):
        stats = subprocess.Popen(['./monitor.sh', BASE_DATA_FOLDER, '01_load_ratings'])
        proc_ok = True

df_rating = spark.read.option("header", True).option("inferSchema", True).csv(BASE_DATA_FOLDER + "ratings.csv")
df_rating = df_rating.drop(df_rating.timestamp)

df_rating.limit(3).toPandas()
if save_stats and proc_ok: stats.kill()

23/06/25 19:31:02 INFO InMemoryFileIndex: It took 1 ms to list leaf files for 1 paths.
23/06/25 19:31:02 INFO InMemoryFileIndex: It took 1 ms to list leaf files for 1 paths.
23/06/25 19:31:02 INFO FileSourceStrategy: Pushed Filters: 
23/06/25 19:31:02 INFO FileSourceStrategy: Post-Scan Filters: (length(trim(value#87, None)) > 0)
23/06/25 19:31:02 INFO MemoryStore: Block broadcast_10 stored as values in memory (estimated size 207.0 KiB, free 433.0 MiB)
23/06/25 19:31:02 INFO MemoryStore: Block broadcast_10_piece0 stored as bytes in memory (estimated size 36.3 KiB, free 432.9 MiB)
23/06/25 19:31:02 INFO BlockManagerInfo: Added broadcast_10_piece0 in memory on brainavlab1.di.univr.it:44791 (size: 36.3 KiB, free: 434.1 MiB)
23/06/25 19:31:02 INFO SparkContext: Created broadcast 10 from csv at NativeMethodAccessorImpl.java:0
23/06/25 19:31:02 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
23/06/25 19:31:02

In [6]:
df_rating.count()

23/06/25 19:31:04 INFO FileSourceStrategy: Pushed Filters: 
23/06/25 19:31:04 INFO FileSourceStrategy: Post-Scan Filters: 
23/06/25 19:31:04 INFO CodeGenerator: Code generated in 6.100637 ms
23/06/25 19:31:04 INFO MemoryStore: Block broadcast_16 stored as values in memory (estimated size 206.9 KiB, free 433.4 MiB)
23/06/25 19:31:04 INFO MemoryStore: Block broadcast_16_piece0 stored as bytes in memory (estimated size 36.3 KiB, free 433.4 MiB)
23/06/25 19:31:04 INFO BlockManagerInfo: Added broadcast_16_piece0 in memory on brainavlab1.di.univr.it:44791 (size: 36.3 KiB, free: 434.2 MiB)
23/06/25 19:31:04 INFO SparkContext: Created broadcast 16 from count at NativeMethodAccessorImpl.java:0
23/06/25 19:31:04 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
23/06/25 19:31:04 INFO DAGScheduler: Registering RDD 49 (count at NativeMethodAccessorImpl.java:0) as input to shuffle 3
23/06/25 19:31:04 INFO DAGSchedule

100836

In [7]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

if save_stats:
    proc_ok = False
    if not os.path.exists(os.path.join('stats', BASE_DATA_FOLDER, '02_prepare_ALS_matrix' + '.csv')):
        stats = subprocess.Popen(['./monitor.sh', BASE_DATA_FOLDER, '02_prepare_ALS_matrix'])
        proc_ok = True
indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in list(set(df_rating.columns)-set(['rating'])) ]
pipeline = Pipeline(stages=indexer)
transformed = pipeline.fit(df_rating).transform(df_rating)
transformed.show()
if save_stats and proc_ok: stats.kill()

23/06/25 19:31:06 INFO FileSourceStrategy: Pushed Filters: 
23/06/25 19:31:06 INFO FileSourceStrategy: Post-Scan Filters: 
23/06/25 19:31:06 INFO CodeGenerator: Code generated in 5.445533 ms
23/06/25 19:31:06 INFO MemoryStore: Block broadcast_19 stored as values in memory (estimated size 206.9 KiB, free 433.1 MiB)
23/06/25 19:31:06 INFO MemoryStore: Block broadcast_19_piece0 stored as bytes in memory (estimated size 36.3 KiB, free 433.1 MiB)
23/06/25 19:31:06 INFO BlockManagerInfo: Added broadcast_19_piece0 in memory on brainavlab1.di.univr.it:44791 (size: 36.3 KiB, free: 434.2 MiB)
23/06/25 19:31:06 INFO SparkContext: Created broadcast 19 from collect at StringIndexer.scala:204
23/06/25 19:31:06 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
23/06/25 19:31:06 INFO DAGScheduler: Registering RDD 57 (collect at StringIndexer.scala:204) as input to shuffle 4
23/06/25 19:31:06 INFO DAGScheduler: Got map s

+------+-------+------+-------------+------------+
|userId|movieId|rating|movieId_index|userId_index|
+------+-------+------+-------------+------------+
|     1|      1|   4.0|         11.0|       111.0|
|     1|      3|   4.0|        422.0|       111.0|
|     1|      6|   4.0|        129.0|       111.0|
|     1|     47|   5.0|         15.0|       111.0|
|     1|     50|   5.0|         14.0|       111.0|
|     1|     70|   3.0|        390.0|       111.0|
|     1|    101|   5.0|       1095.0|       111.0|
|     1|    110|   4.0|          7.0|       111.0|
|     1|    151|   5.0|        534.0|       111.0|
|     1|    157|   5.0|       1991.0|       111.0|
|     1|    163|   5.0|        280.0|       111.0|
|     1|    216|   5.0|        451.0|       111.0|
|     1|    223|   3.0|        121.0|       111.0|
|     1|    231|   5.0|         63.0|       111.0|
|     1|    235|   4.0|        254.0|       111.0|
|     1|    260|   5.0|          5.0|       111.0|
|     1|    296|   3.0|        

23/06/25 19:31:07 INFO CodeGenerator: Code generated in 5.147759 ms


In [8]:
if save_stats:
    proc_ok = False
    if not os.path.exists(os.path.join('stats', BASE_DATA_FOLDER, '03_train_ALS_model' + '.csv')):
        stats = subprocess.Popen(['./monitor.sh', BASE_DATA_FOLDER, '03_train_ALS_model'])
        proc_ok = True
epochs = 5
als=ALS(maxIter=epochs,
        regParam=0.09,
        rank=25,
        userCol="userId_index",
        itemCol="movieId_index",
        ratingCol="rating",
        coldStartStrategy="drop",
        nonnegative=True)
model=als.fit(transformed)
if save_stats and proc_ok: stats.kill()

23/06/25 19:31:09 INFO FileSourceStrategy: Pushed Filters: 
23/06/25 19:31:09 INFO FileSourceStrategy: Post-Scan Filters: 
23/06/25 19:31:09 INFO CodeGenerator: Code generated in 12.85697 ms
23/06/25 19:31:09 INFO MemoryStore: Block broadcast_27 stored as values in memory (estimated size 206.9 KiB, free 432.6 MiB)
23/06/25 19:31:09 INFO MemoryStore: Block broadcast_27_piece0 stored as bytes in memory (estimated size 36.3 KiB, free 432.6 MiB)
23/06/25 19:31:09 INFO BlockManagerInfo: Added broadcast_27_piece0 in memory on brainavlab1.di.univr.it:44791 (size: 36.3 KiB, free: 434.0 MiB)
23/06/25 19:31:09 INFO SparkContext: Created broadcast 27 from rdd at ALS.scala:727
23/06/25 19:31:09 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
23/06/25 19:31:09 INFO Instrumentation: [9e12e313] Stage class: ALS
23/06/25 19:31:09 INFO Instrumentation: [9e12e313] Stage uid: ALS_21c5071f5d2f
23/06/25 19:31:09 INFO FileS

In [9]:
model.itemFactors.count()

23/06/25 19:31:13 INFO CodeGenerator: Code generated in 6.978218 ms
23/06/25 19:31:13 INFO DAGScheduler: Registering RDD 202 (count at NativeMethodAccessorImpl.java:0) as input to shuffle 19
23/06/25 19:31:13 INFO DAGScheduler: Got map stage job 21 (count at NativeMethodAccessorImpl.java:0) with 10 output partitions
23/06/25 19:31:13 INFO DAGScheduler: Final stage: ShuffleMapStage 69 (count at NativeMethodAccessorImpl.java:0)
23/06/25 19:31:13 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 60, ShuffleMapStage 68)
23/06/25 19:31:13 INFO DAGScheduler: Missing parents: List()
23/06/25 19:31:13 INFO DAGScheduler: Submitting ShuffleMapStage 69 (MapPartitionsRDD[202] at count at NativeMethodAccessorImpl.java:0), which has no missing parents
23/06/25 19:31:13 INFO MemoryStore: Block broadcast_47 stored as values in memory (estimated size 361.0 KiB, free 431.3 MiB)
23/06/25 19:31:13 INFO MemoryStore: Block broadcast_47_piece0 stored as bytes in memory (estimated size 189.4 KiB

9724

In [10]:
if save_stats:
    proc_ok = False
    if not os.path.exists(os.path.join('stats', BASE_DATA_FOLDER, '04_eval_ALS_model' + '.csv')):
        stats = subprocess.Popen(['./monitor.sh', BASE_DATA_FOLDER, '04_eval_ALS_model'])
        proc_ok = True
evaluator=RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")
predictions=model.transform(transformed)
rmse=evaluator.evaluate(predictions)
print("RMSE="+str(rmse))
predictions.show()
if save_stats and proc_ok: stats.kill()

23/06/25 19:31:14 INFO FileSourceStrategy: Pushed Filters: 
23/06/25 19:31:14 INFO FileSourceStrategy: Post-Scan Filters: 
23/06/25 19:31:14 INFO CodeGenerator: Code generated in 6.059151 ms
23/06/25 19:31:14 INFO MemoryStore: Block broadcast_49 stored as values in memory (estimated size 206.9 KiB, free 430.9 MiB)
23/06/25 19:31:14 INFO MemoryStore: Block broadcast_49_piece0 stored as bytes in memory (estimated size 36.3 KiB, free 430.9 MiB)
23/06/25 19:31:14 INFO BlockManagerInfo: Added broadcast_49_piece0 in memory on brainavlab1.di.univr.it:44791 (size: 36.3 KiB, free: 432.4 MiB)
23/06/25 19:31:14 INFO SparkContext: Created broadcast 49 from rdd at RegressionEvaluator.scala:125
23/06/25 19:31:14 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
23/06/25 19:31:14 INFO DAGScheduler: Registering RDD 209 (rdd at RegressionEvaluator.scala:125) as input to shuffle 20
23/06/25 19:31:14 INFO DAGScheduler: Got

RMSE=0.5637394489922718


23/06/25 19:31:15 INFO ShuffleBlockFetcherIterator: Getting 10 (1209.3 KiB) non-empty blocks including 10 (1209.3 KiB) local and 0 (0.0 B) host-local and 0 (0.0 B) push-merged-local and 0 (0.0 B) remote blocks
23/06/25 19:31:15 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
23/06/25 19:31:15 INFO Executor: Finished task 0.0 in stage 151.0 (TID 217). 965718 bytes result sent to driver
23/06/25 19:31:15 INFO TaskSetManager: Finished task 0.0 in stage 151.0 (TID 217) in 15 ms on brainavlab1.di.univr.it (executor driver) (1/1)
23/06/25 19:31:15 INFO TaskSchedulerImpl: Removed TaskSet 151.0, whose tasks have all completed, from pool 
23/06/25 19:31:15 INFO DAGScheduler: ResultStage 151 ($anonfun$withThreadLocalCaptured$1 at FutureTask.java:264) finished in 0.019 s
23/06/25 19:31:15 INFO DAGScheduler: Job 33 is finished. Cancelling potential speculative or zombie tasks for this job
23/06/25 19:31:15 INFO TaskSchedulerImpl: Killing all running tasks in stage 151: Stage fin

+------+-------+------+-------------+------------+----------+
|userId|movieId|rating|movieId_index|userId_index|prediction|
+------+-------+------+-------------+------------+----------+
|   279|      1|   3.0|         11.0|       148.0| 3.5241723|
|   279|     50|   3.5|         14.0|       148.0| 3.9655216|
|   279|    104|   3.5|        138.0|       148.0| 2.9611485|
|   279|    110|   4.0|          7.0|       148.0|  4.039477|
|   279|    260|   5.0|          5.0|       148.0|  4.078675|
|   279|    296|   5.0|          2.0|       148.0| 4.1060195|
|   279|    318|   4.5|          1.0|       148.0| 4.0075645|
|   279|    356|   3.0|          0.0|       148.0|  3.785623|
|   279|    377|   3.5|         34.0|       148.0|  3.096514|
|   279|    480|   4.0|          6.0|       148.0| 3.6302817|
|   279|    541|   5.0|         79.0|       148.0| 3.8230996|
|   279|    588|   2.0|         27.0|       148.0|   3.25816|
|   279|    589|   4.0|          8.0|       148.0| 3.8712645|
|   279|

23/06/25 19:31:15 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
23/06/25 19:31:15 INFO DAGScheduler: Got job 34 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
23/06/25 19:31:15 INFO DAGScheduler: Final stage: ResultStage 153 (showString at NativeMethodAccessorImpl.java:0)
23/06/25 19:31:15 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 152)
23/06/25 19:31:15 INFO DAGScheduler: Missing parents: List()
23/06/25 19:31:15 INFO DAGScheduler: Submitting ResultStage 153 (MapPartitionsRDD[244] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
23/06/25 19:31:15 INFO MemoryStore: Block broadcast_66 stored as values in memory (estimated size 45.1 KiB, free 421.3 MiB)
23/06/25 19:31:15 INFO BlockManagerInfo: Removed broadcast_60_piece0 on brainavlab1.di.univr.it:44791 in memory (size: 190.3 KiB, free: 430.1 MiB)
23/06/25 19:31:15 INFO MemoryStore: Block broadcast_66_piece0 stored as bytes in m

In [11]:
from pyspark.sql.functions import lit, current_timestamp, unix_timestamp
ver = model.uid
ts = unix_timestamp(current_timestamp())
movie_vectors = model.itemFactors.select("id",\
                                         col("features").alias("model_factor"),\
                                         lit(ver).alias("model_version"),\
                                         ts.alias("model_timestamp"))
movie_vectors.show(2)

+---+--------------------+----------------+---------------+
| id|        model_factor|   model_version|model_timestamp|
+---+--------------------+----------------+---------------+
|  0|[0.24228135, 0.62...|ALS_21c5071f5d2f|     1687714275|
| 10|[0.40081677, 0.91...|ALS_21c5071f5d2f|     1687714275|
+---+--------------------+----------------+---------------+
only showing top 2 rows



23/06/25 19:31:15 INFO CodeGenerator: Code generated in 7.749827 ms
23/06/25 19:31:15 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
23/06/25 19:31:15 INFO DAGScheduler: Got job 35 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
23/06/25 19:31:15 INFO DAGScheduler: Final stage: ResultStage 166 (showString at NativeMethodAccessorImpl.java:0)
23/06/25 19:31:15 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 165, ShuffleMapStage 155)
23/06/25 19:31:15 INFO DAGScheduler: Missing parents: List()
23/06/25 19:31:15 INFO DAGScheduler: Submitting ResultStage 166 (MapPartitionsRDD[247] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
23/06/25 19:31:15 INFO MemoryStore: Block broadcast_67 stored as values in memory (estimated size 363.4 KiB, free 427.4 MiB)
23/06/25 19:31:15 INFO MemoryStore: Block broadcast_67_piece0 stored as bytes in memory (estimated size 189.8 KiB, free 427.2 MiB)
23/06/25

In [12]:
movieId_index_meta = [
    f.metadata for f in transformed.schema.fields if f.name == "movieId_index"]
movieId_index_labels = movieId_index_meta[0]["ml_attr"]["vals"]

from pyspark.ml.feature import IndexToString

reviewerId_converter = IndexToString(inputCol="id", outputCol="movieId",   labels=movieId_index_labels)
PredictedLabels = reviewerId_converter.transform(movie_vectors)
PredictedLabels = PredictedLabels.drop('id')
PredictedLabels.show(10)

+--------------------+----------------+---------------+-------+
|        model_factor|   model_version|model_timestamp|movieId|
+--------------------+----------------+---------------+-------+
|[0.24228135, 0.62...|ALS_21c5071f5d2f|     1687714277|    356|
|[0.40081677, 0.91...|ALS_21c5071f5d2f|     1687714277|   2959|
|[0.20778744, 0.29...|ALS_21c5071f5d2f|     1687714277|   1210|
|[0.052279662, 0.4...|ALS_21c5071f5d2f|     1687714277|    380|
|[0.007815801, 0.5...|ALS_21c5071f5d2f|     1687714277|    344|
|[0.0, 0.5564098, ...|ALS_21c5071f5d2f|     1687714277|   1265|
|[0.0, 0.3109011, ...|ALS_21c5071f5d2f|     1687714277|   1136|
|[0.4565808, 0.631...|ALS_21c5071f5d2f|     1687714277|   6874|
|[0.15146497, 0.70...|ALS_21c5071f5d2f|     1687714277|   4995|
|[0.024200078, 0.7...|ALS_21c5071f5d2f|     1687714277|   1073|
+--------------------+----------------+---------------+-------+
only showing top 10 rows



23/06/25 19:31:17 INFO CodeGenerator: Code generated in 8.524313 ms
23/06/25 19:31:17 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
23/06/25 19:31:17 INFO DAGScheduler: Got job 36 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
23/06/25 19:31:17 INFO DAGScheduler: Final stage: ResultStage 179 (showString at NativeMethodAccessorImpl.java:0)
23/06/25 19:31:17 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 168, ShuffleMapStage 178)
23/06/25 19:31:17 INFO DAGScheduler: Missing parents: List()
23/06/25 19:31:17 INFO DAGScheduler: Submitting ResultStage 179 (MapPartitionsRDD[250] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
23/06/25 19:31:17 INFO MemoryStore: Block broadcast_68 stored as values in memory (estimated size 438.3 KiB, free 426.8 MiB)
23/06/25 19:31:17 INFO MemoryStore: Block broadcast_68_piece0 stored as bytes in memory (estimated size 240.7 KiB, free 426.6 MiB)
23/06/25

In [13]:
PredictedLabels.count()

23/06/25 19:31:18 INFO DAGScheduler: Registering RDD 253 (count at NativeMethodAccessorImpl.java:0) as input to shuffle 26
23/06/25 19:31:18 INFO DAGScheduler: Got map stage job 37 (count at NativeMethodAccessorImpl.java:0) with 10 output partitions
23/06/25 19:31:18 INFO DAGScheduler: Final stage: ShuffleMapStage 192 (count at NativeMethodAccessorImpl.java:0)
23/06/25 19:31:18 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 183, ShuffleMapStage 191)
23/06/25 19:31:18 INFO DAGScheduler: Missing parents: List()
23/06/25 19:31:18 INFO DAGScheduler: Submitting ShuffleMapStage 192 (MapPartitionsRDD[253] at count at NativeMethodAccessorImpl.java:0), which has no missing parents
23/06/25 19:31:18 INFO MemoryStore: Block broadcast_69 stored as values in memory (estimated size 361.0 KiB, free 426.2 MiB)
23/06/25 19:31:18 INFO MemoryStore: Block broadcast_69_piece0 stored as bytes in memory (estimated size 189.4 KiB, free 426.0 MiB)
23/06/25 19:31:18 INFO BlockManagerInfo: Added

9724

In [14]:
from elasticsearch import Elasticsearch

# test your ES instance is running
es = Elasticsearch("htts://localhost:9200")
es.info(pretty=True)

ObjectApiResponse({'name': 'ilaria-MS-7B94', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'i-HQNtSZToi8ekX-DUdoBg', 'version': {'number': '8.8.0', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': 'c01029875a091076ed42cdb3a41c10b1a9a5a20f', 'build_date': '2023-05-23T17:16:07.179039820Z', 'build_snapshot': False, 'lucene_version': '9.6.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [15]:
if es.indices.exists(index="movies"):
    es.indices.delete(index="movies")
VECTOR_DIM = 25

movies_mapping = {
    # this mapping definition sets up the metadata fields for the movies
    "mappings": {
        "properties": {
            "movieId": {
                "type": "keyword"
            },
            "title": {
                "type": "keyword"
            },
            "genres": {
                "type": "keyword"
            },
            "main_genre": {
                "type": "keyword"
            },
            # the following fields define our model factor vectors and metadata
            "model_factor": {
                "type": "dense_vector",
                "dims" : VECTOR_DIM
            },
            "model_version": {
                "type": "keyword"
            },
            "model_timestamp": {
                "type": "date"
            }          
        }
    }
}

res_movies = es.indices.create(index="movies", body=movies_mapping)

print("Created indices:")
print(res_movies)

Created indices:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'movies'}


/tmp/ipykernel_1380295/201502410.py:36: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res_movies = es.indices.create(index="movies", body=movies_mapping)


In [16]:
movies_data.write.format("es").option("es.mapping.id", "movieId").save("movies")
num_movies_df = movies_data.count()
num_movies_es = es.count(index="movies")['count']
# check load went ok
print("Movies DF count: {}".format(num_movies_df))
print("ES index count: {}".format(num_movies_es))

23/06/25 19:31:22 INFO FileSourceStrategy: Pushed Filters: 
23/06/25 19:31:22 INFO FileSourceStrategy: Post-Scan Filters: 
23/06/25 19:31:22 INFO MemoryStore: Block broadcast_71 stored as values in memory (estimated size 206.8 KiB, free 425.8 MiB)
23/06/25 19:31:22 INFO MemoryStore: Block broadcast_71_piece0 stored as bytes in memory (estimated size 36.3 KiB, free 425.8 MiB)
23/06/25 19:31:22 INFO BlockManagerInfo: Added broadcast_71_piece0 in memory on brainavlab1.di.univr.it:44791 (size: 36.3 KiB, free: 431.1 MiB)
23/06/25 19:31:22 INFO SparkContext: Created broadcast 71 from rdd at EsSparkSQL.scala:103
23/06/25 19:31:22 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
23/06/25 19:31:22 INFO DAGScheduler: Registering RDD 261 (rdd at EsSparkSQL.scala:103) as input to shuffle 27
23/06/25 19:31:22 INFO DAGScheduler: Got map stage job 39 (rdd at EsSparkSQL.scala:103) with 1 output partitions
23/06/25 19:3

Movies DF count: 9742
ES index count: 9742


23/06/25 19:31:23 INFO Executor: Finished task 0.0 in stage 209.0 (TID 233). 4535 bytes result sent to driver
23/06/25 19:31:23 INFO TaskSetManager: Finished task 0.0 in stage 209.0 (TID 233) in 971 ms on brainavlab1.di.univr.it (executor driver) (1/1)
23/06/25 19:31:23 INFO TaskSchedulerImpl: Removed TaskSet 209.0, whose tasks have all completed, from pool 
23/06/25 19:31:23 INFO DAGScheduler: ResultStage 209 (runJob at EsSparkSQL.scala:103) finished in 0.979 s
23/06/25 19:31:23 INFO DAGScheduler: Job 40 is finished. Cancelling potential speculative or zombie tasks for this job
23/06/25 19:31:23 INFO TaskSchedulerImpl: Killing all running tasks in stage 209: Stage finished
23/06/25 19:31:23 INFO DAGScheduler: Job 40 finished: runJob at EsSparkSQL.scala:103, took 0.983207 s
23/06/25 19:31:23 INFO FileSourceStrategy: Pushed Filters:                      
23/06/25 19:31:23 INFO FileSourceStrategy: Post-Scan Filters: 
23/06/25 19:31:23 INFO HashAggregateExec: spark.sql.codegen.aggregate.m

In [17]:
es.search(index="movies", q="genres:Drama", size=3)

ObjectApiResponse({'took': 38, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 4361, 'relation': 'eq'}, 'max_score': 1.0419514, 'hits': [{'_index': 'movies', '_id': '100', '_score': 1.0419514, '_source': {'movieId': '100', 'title': 'City Hall (1996)', 'genres': ['Drama', 'Thriller']}}, {'_index': 'movies', '_id': '100226', '_score': 1.0419514, '_source': {'movieId': '100226', 'title': 'Why Stop Now (2012)', 'genres': ['Comedy', 'Drama']}}, {'_index': 'movies', '_id': '100277', '_score': 1.0419514, '_source': {'movieId': '100277', 'title': 'Tabu (2012)', 'genres': ['Drama', 'Romance']}}]}})

In [18]:
PredictedLabels.write.format("es") \
    .option("es.mapping.id", "movieId") \
    .option("es.write.operation", "upsert") \
    .save("movies", mode="append")

23/06/25 19:31:25 INFO CodeGenerator: Code generated in 8.635506 ms
23/06/25 19:31:25 INFO SparkContext: Starting job: runJob at EsSparkSQL.scala:103
23/06/25 19:31:25 INFO DAGScheduler: Got job 44 (runJob at EsSparkSQL.scala:103) with 10 output partitions
23/06/25 19:31:25 INFO DAGScheduler: Final stage: ResultStage 228 (runJob at EsSparkSQL.scala:103)
23/06/25 19:31:25 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 227, ShuffleMapStage 217)
23/06/25 19:31:25 INFO DAGScheduler: Missing parents: List()
23/06/25 19:31:25 INFO DAGScheduler: Submitting ResultStage 228 (MapPartitionsRDD[282] at rdd at EsSparkSQL.scala:103), which has no missing parents
23/06/25 19:31:25 INFO MemoryStore: Block broadcast_78 stored as values in memory (estimated size 460.9 KiB, free 424.9 MiB)
23/06/25 19:31:25 INFO MemoryStore: Block broadcast_78_piece0 stored as bytes in memory (estimated size 245.9 KiB, free 424.6 MiB)
23/06/25 19:31:25 INFO BlockManagerInfo: Added broadcast_78_piece0 in 

In [19]:
import pandas as pd

def vector_query(query_vec, genres, vector_field, cosine=False, use_genre=True):
    """
    Construct an Elasticsearch script score query using `dense_vector` fields
    
    The script score query takes as parameters the query vector (as a Python list)
    
    Parameters
    ----------
    query_vec : list
        The query vector
    vector_field : str
        The field name in the document against which to score `query_vec`
    q : str, optional
        Query string for the search query (default: '*' to search across all documents)
    cosine : bool, optional
        Whether to compute cosine similarity. If `False` then the dot product is computed (default: False)
     
    Note: Elasticsearch cannot rank negative scores. Therefore, in the case of the dot product, a sigmoid transform
    is applied. In the case of cosine similarity, 1.0 is added to the score. In both cases, documents with no 
    factor vectors are ignored by applying a 0.0 score.
    
    The query vector passed in will be the user factor vector (if generating recommended items for a user)
    or product factor vector (if generating similar items for a given item)
    """
    
    if cosine:
        score_fn = "doc['{v}'].size() == 0 ? 0 : cosineSimilarity(params.vector, '{v}') + 1.0"
    else:
        score_fn = "doc['{v}'].size() == 0 ? 0 : sigmoid(1, Math.E, -dotProduct(params.vector, '{v}'))"
       
    score_fn = score_fn.format(v=vector_field, fn=score_fn)
    
    if not use_genre:
        return {
            "query": {
                "script_score": {
                    "query" : { 
                        "match_all": {}
                    },
                    "script": {
                        "source": score_fn,
                        "params": {
                            "vector": query_vec
                        }
                    }
                }
            }
        }

    return {
        "query": {
            "script_score": {
                "query" : { 
                    "bool" : {
                        "filter" : {
                            "terms" : {
                                "genres" : genres
                            }
                        }
                    }
                },
                "script": {
                    "source": score_fn,
                        "params": {
                            "vector": query_vec
                        }
                }
            }
        }
    }



def get_similar(the_id, num=10, index="movies", vector_field='model_factor'):
    """
    Given a item id, execute the recommendation script score query to find similar items,
    ranked by cosine similarity. We return the `num` most similar, excluding the item itself.
    """
    response = es.get(index=index, id=the_id)
    src = response['_source']
    if vector_field in src:
        query_vec = src[vector_field]
        genres = src['genres']
        q = vector_query(query_vec, genres, vector_field, cosine=True)
#         print(q)
        results = es.search(index=index, body=q)
        hits = results['hits']['hits']
        return src,hits[1:num+1]

def display_similar(the_id, num=10, es_index="movies"):
    """
    Display query product, together with similar product and similarity scores, in a table
    """
    movie, recs = get_similar(the_id, num, es_index)
       
    display(HTML("<h2>Get similar movies for:</h2>"))
    display(HTML("<h4>%s (movieId - %s) (genres - %s)</h4>" % (movie['title'], movie['movieId'], movie['genres'])))
    display(HTML("<br>"))
    display(HTML("<h2>People who liked this movie also liked these:</h2>"))
    sim_html = "<table border=0>"
    i = 0
    pd_data = []
    for rec in recs:
        r_score = rec['_score']
        r_title = rec['_source']['title']
        r_genres = rec['_source']['genres']
        r = {}
        r['movieId'] = rec['_source']['movieId']
        r['title'] = r_title
        r['score'] = r_score
        r['genres'] = r_genres
        pd_data.append(r)
        # r_im_url = rec['_source']['image']
        sim_html += "<tr><td><h5>%s</h5><img src=%s width=150></img></td><td><h5>%2.3f</h5></td></tr>" % (r_title, "fake img", r_score)
        i += 1
    sim_html += "</table>"
    pd.set_option('display.max_colwidth', 5) 
    pd_df = pd.DataFrame (pd_data)
    display(HTML(pd_df.to_html()))
    # display(HTML(sim_html))


In [20]:
from IPython.display import Image, HTML, display
data = display_similar('2324', num=10)

/tmp/ipykernel_1380295/2686027687.py:88: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  results = es.search(index=index, body=q)


,movieId,title,score,genres
0,2231,Rounders (1998),1.944854,[Drama]
1,8983,House of Flying Daggers (Shi mian mai fu) (2004),1.940879,"[Action, Drama, Romance]"
2,33903,"Edukators, The (Die Fetten Jahre sind vorbei) (2004)",1.934960,"[Comedy, Crime, Drama, Romance]"
3,2131,Autumn Sonata (Höstsonaten) (1978),1.932567,[Drama]
4,2291,Edward Scissorhands (1990),1.932564,"[Drama, Fantasy, Romance]"
5,7361,Eternal Sunshine of the Spotless Mind (2004),1.930569,"[Drama, Romance, Sci-Fi]"
6,26693,It (1990),1.928796,"[Drama, Fantasy, Horror, Mystery, Thriller]"
7,6210,Volcano High (Whasango) (2001),1.928796,"[Action, Comedy]"
8,2329,American History X (1998),1.927934,"[Crime, Drama]"
